# GitHub

## Git Commit and Push

In [2]:
%cd "/content/drive/My Drive/Projects/QAMP Spring 2023/qamp-spring-23"

/content/drive/My Drive/Projects/QAMP Spring 2023/qamp-spring-23


In [3]:
# Check status
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    Untitled

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Development Notebook.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
# Add/track changed files

# only modified files
#!git add -u

# all files
!git add .

In [ ]:
# Commit the changes
# DON'T FORGET TO CHANGE THE COMMIT MESSAGE
safety_net = input('Have you changed the Commit Message? (Y/N)')
if safety_net == 'Y':
  !git commit -m "Add notebook." # commit message
  !git config --global user.email "eraraya.ricardo19@gmail.com"
  !git config --global user.name "ericardomuten"
else:
  print('Please change the Commit Message first.')

Have you changed the Commit Message? (Y/N)Y
[main dbc67b2] Add notebook.
 1 file changed, 1 insertion(+)
 create mode 100644 Untitled


In [ ]:
# Push changes to GitHub Repo
!git push

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 521 bytes | 86.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To https://github.com/ericardomuten/qamp-spring-23.git
   146e330..dbc67b2  main -> main


In [ ]:
# Check status
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


## Git Pull

In [ ]:
%cd "/content/drive/My Drive/Projects/QAMP Spring 2023/qamp-spring-23"

/content/drive/My Drive/Projects/QAMP Spring 2023/qamp-spring-23


In [ ]:
!git pull

Already up to date.


In [ ]:
# Check status
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
